In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import os
import json
from scipy.stats import t
from datetime import datetime, timedelta

In [ ]:
path = 'C:/Users/Janet/OneDrive - The University of Chicago/Data_policy/final-project-janet'

file_name = 'output.csv'
file_path = os.path.join(path, file_name)
df = pd.read_csv(file_path)

In [ ]:
event_date = datetime(2018, 6, 28)
estimation_window_start = event_date - timedelta(days=365)
estimation_window_end = event_date - timedelta(days=30)
event_window_start = event_date - timedelta(days=10)
event_window_end = event_date + timedelta(days=10)
stock_list = df['Ticker'].unique()
# stock_list = ['ATUS', 'AFTM', 'ATUS','AAPL']  # List of stocks to analyze



In [ ]:

combined_data = pd.DataFrame()

for ticker in stock_list:
    try:
        stock = yf.Ticker(ticker)
        
        # Fetch historical stock data
        historical_data = stock.history(start=estimation_window_start, end=event_window_end)
        if historical_data.empty:
            print(f"No data for {ticker} for specified dates.")
            continue

        historical_data['Ticker'] = ticker

        info = stock.info
        market_cap = info.get('marketCap', 'N/A')
        historical_data['MarketCap'] = market_cap

        # Append this stock's data to the combined DataFrame
        combined_data = pd.concat([combined_data, historical_data])
    except Exception as e:
        print(f"Error processing {ticker}: {e}")



In [52]:
unique_ticker_count = combined_data['Ticker'].nunique()
print(unique_ticker_count)
print(combined_data['Dividends'].nunique())
print(combined_data['Volume'].nunique())

258
98
19663


In [53]:
sp500_path = r'C:\Users\Janet\OneDrive - The University of Chicago\Data_policy\final-project-janet\data'
json_file_name = 'sp500_constituents.json'
json_file_path = os.path.join(sp500_path, json_file_name)
with open(json_file_path, 'r') as file:
    sp500_data = json.load(file)
sp500_tickers = sp500_data.get("2018/06/20", []) #extract sp500 constitutes list at the most recent date of event_date (2018/6/28)
combined_data['sp500'] = combined_data['Ticker'].apply(lambda x: 1 if x in sp500_tickers else 0)

#drop variables
columns_to_drop = ['Stock Splits', 'Capital Gains']

combined_data = combined_data.drop(columns=columns_to_drop, axis=1)
combined_data.to_csv('combined_stock_data.csv')

In [47]:
sp500_counts = combined_data['sp500'].value_counts()
count_sp500_is_1 = sp500_counts.get(1, 0)
print("Number of instances where sp500 equals 1:", count_sp500_is_1)

Number of instances where sp500 equals 1: 3870
